## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 128
num_epoch  = 180

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0001

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [60, 120, 150]

pretrained_model       = './cifar100_resnet83_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Resnet83 model

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarResnetConfig(num_layers = 83, cifar = 100, block_type = 'Bottleneck')
model = model_gen.GetCifarResnet()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 4.293037, Test Acc: 6.59 %
2 epoch end, loss: 3.682521, Test Acc: 16.86 %
3 epoch end, loss: 3.166202, Test Acc: 25.57 %
4 epoch end, loss: 2.736031, Test Acc: 27.10 %
5 epoch end, loss: 2.458947, Test Acc: 31.71 %
6 epoch end, loss: 2.253214, Test Acc: 37.72 %
7 epoch end, loss: 2.094925, Test Acc: 42.81 %
8 epoch end, loss: 1.959650, Test Acc: 43.64 %
9 epoch end, loss: 1.834413, Test Acc: 43.82 %
10 epoch end, loss: 1.728676, Test Acc: 48.84 %
11 epoch end, loss: 1.637841, Test Acc: 50.28 %
12 epoch end, loss: 1.542297, Test Acc: 51.52 %
13 epoch end, loss: 1.477364, Test Acc: 51.81 %
14 epoch end, loss: 1.410416, Test Acc: 54.16 %
15 epoch end, loss: 1.351602, Test Acc: 55.66 %
16 epoch end, loss: 1.301559, Test Acc: 54.37 %
17 epoch end, loss: 1.256773, Test Acc: 55.25 %
18 epoch end, loss: 1.213158, Test Acc: 56.79 %
19 epoch end, loss: 1.175079, Test Acc: 57.26 %
20 epoch end, loss: 1.139463, Test Acc: 55.88 %
21 epoch end, loss: 1.112977, Test Acc: 57.08 %
22

171 epoch end, loss: 0.017346, Test Acc: 72.83 %
172 epoch end, loss: 0.017488, Test Acc: 72.50 %
173 epoch end, loss: 0.017793, Test Acc: 72.77 %
174 epoch end, loss: 0.017714, Test Acc: 72.57 %
175 epoch end, loss: 0.016943, Test Acc: 72.58 %
176 epoch end, loss: 0.017891, Test Acc: 72.49 %
177 epoch end, loss: 0.017586, Test Acc: 72.58 %
178 epoch end, loss: 0.017030, Test Acc: 72.41 %
179 epoch end, loss: 0.017364, Test Acc: 72.69 %
180 epoch end, loss: 0.017226, Test Acc: 72.80 %

Training is finished!


In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 72.80 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)